# C.V. Agents

1. Analyze a job posting to extract structured requirements.
2. Profile a candidate by querying a knowledge base for data relevant to the job posting
3. Optimize the candidate's CV based on the job requirements and profile data

## Data Flow

1. Job posting URL → Job Analyst → Structured job requirements
2. Candidate data + Job requirements → Candidate Profiler → Profile analysis  
3. Profile + Job requirements → CV Strategist → Optimization recommendations
4. Optimized data → Template engine → LaTeX → PDF

*Import the agents and tasks:*

In [ ]:
from optimizer.agents.job_analyst import job_analyst as analyst
from optimizer.agents.candidate_profiler import candidate_profiler as profiler
from optimizer.agents.cv_strategist import cv_strategist as strategist

from optimizer.tasks.job_analysis_task import create_job_analysis_task
from optimizer.tasks.candidate_profile_task import create_candidate_profile_task
from optimizer.tasks.cv_optimizing_task import create cv_optimizing_task

*Chunk knowledge base docs and embed them in a vector store:*

In [ ]:
from embedding_tool import KnowledgeBaseEmbedder
embedder = KnowledgeBaseEmbedder()
result = embedder.create_embeddings()

*Configure the Crew AI environment:*

In [ ]:
from crewai import Crew

inputs = {
    "job_posting_url": "https://app.welcometothejungle.com/dashboard/jobs/oA1SArxV",
}